<a href="https://colab.research.google.com/github/kasri-mids/MovieGenreClassification/blob/master/src/Data/4_Extract_Bert_Tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting BERT Tokens for train, dev and val

In this notebook, we take our cleaned dataset and generate BERT tokens from the movie plot summaries which are then split into a train, dev and validation set and we save those files for later use.

In [0]:
!pip install tensorflow==2.1.0

     |████████████████████████████████| 421.8MB 36kB/s 
     |████████████████████████████████| 3.9MB 45.3MB/s 
     |████████████████████████████████| 450kB 45.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=b8d19f314717ba6247ac7b21de871823c47a388b4646b034599bb7c484974ff7
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc

In [0]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [0]:
# Imports

import json
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
import tensorflow_hub as hub
from time import time
import io
import re
from csv import reader
from datetime import datetime
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Lambda
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.backend import sparse_categorical_crossentropy
from tensorflow.keras.layers import Dense, TimeDistributed

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
# Installing relevant modules
!pip install bert-for-tf2
!pip install sentencepiece

In [0]:
# Define maximal length of input 'sentences' (post tokenization).
max_length = 128

# ERROR TEST
ERROR_TEST = 1

In [0]:
# Here, only have to run once because we are tokenizing the movieplot which is same regardless of NUM_LABELS
if (ERROR_TEST):
    NUM_LABELS = 1
    
    df_train = pd.read_csv('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_df_train_labels_' + str(NUM_LABELS) + '.csv')
    df_dev = pd.read_csv('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_df_dev_labels_' + str(NUM_LABELS) + '.csv')
    df_val = pd.read_csv('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_df_val_labels_' + str(NUM_LABELS) + '.csv')
    
    data_save_prefix = '/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/baseline_test/'
    model_save_prefix = '/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/model/baseline_test/'
    plot_save_prefix = '/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/plots/baseline_test/'

else:
    NUM_LABELS = 10
    
    df_train = pd.read_csv('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_df_train_labels_' + str(NUM_LABELS) + '.csv')
    df_dev = pd.read_csv('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_df_dev_labels_' + str(NUM_LABELS) + '.csv')
    df_val = pd.read_csv('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_df_val_labels_' + str(NUM_LABELS) + '.csv')

    data_save_prefix = '/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/baseline/'
    model_save_prefix = '/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/model/baseline/'
    plot_save_prefix = '/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/plots/baseline/'


In [0]:
df_train.head()

,movieid_poster,indexes,movieid,movieplot,labels
0,tt1786530.jpg,73777,tt1786530.jpg,When a mysterious but attractive stranger appl...,[0]
1,tt0146592.jpg,10645,tt0146592.jpg,"A lonely obese nurse, working at a hospital te...",[1]
2,tt0088153.jpg,2041,tt0088153.jpg,Doc Jenkins is a singer/songwriter who tries t...,[1]
3,tt3733098.jpg,79291,tt3733098.jpg,Rosa is a mature police officer with both a ga...,[1]
4,tt1657507.jpg,59996,tt1657507.jpg,In early-1990s Bogotá--after witnessing the gr...,[1]


In [0]:
import bert

# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/1"

def create_tokenizer():
    """Get the vocab file and casing info from the Hub module."""
    bert_layer = hub.KerasLayer(BERT_MODEL_HUB)
    vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
    tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case)
    return tokenizer

tokenizer = create_tokenizer()

In [0]:
def addWord(word):
    """
    Convert a word into a word token. Note that the word can be tokenized to two or more tokens. 
        
    arguments: word, pos label, ner label
    returns: dictionary with tokens and labels
    """
    # the dataset contains various '"""' combinations which we choose to truncate to '"', etc. 
    if word == '""""':
        word = '"'
    elif word == '``':
        word = '`'
        
    tokens = tokenizer.tokenize(word)
    tokenLength = len(tokens) # find number of tokens corresponfing to word to later add 'X' tokens to labels
    
    addDict = dict()
    
    addDict['wordToken'] = tokens
    addDict['tokenLength'] = tokenLength
    
    return addDict

In [0]:
def construct_bert_sentences(df, plotCol, max_length = 128):
    """
        Read the file line by line and construct sentences. A sentence end is marked by the word 'sentence' in the next row.
        You need to take care of that. Also, you need to cap sentence length using max_length. Sentences which are shorter than 
        max_length need to be padded. Also, we choose to end all sentences with a [SEP] token, padded or not. 
    """

    # lists for sentences, tokens, labels, etc.  
    sentenceList = []
    sentenceTokenList = []
    sentLengthList = []

    # lists for BERT input
    bertSentenceIDs = []
    bertMasks = []
    bertSequenceIDs = []
    # genreLabels = []
    ctr = 0

    for index,rows in df.iterrows():
        ctr += 1

        sentence = ''
        sum_plot = rows[plotCol]
        # genreLabels.append((rows['labels']))
        sent = re.sub(r'(?!(([^"]*"){2})*[^"]*$),', '', sum_plot)  # deal with '"10,000"' and convert them to '10000' -> TO DO : Is this working?
        words = sent.split()
        sentenceTokens = ['[CLS]']
        sentenceLength = len(words)
        for word in words:
            sentence += ' ' + word
            addDict = addWord(word)
            sentenceTokens += addDict['wordToken']
            # Create space for at least a final '[SEP]' token

        sentenceLength = len(sentenceTokens)
        if sentenceLength >= max_length - 1: 
            sentenceTokens = sentenceTokens[:max_length - 2]

        sentenceLength = min(max_length -1, len(sentenceTokens))
        sentenceTokens += ['[SEP]'] + ['[PAD]'] * (max_length -1 - len(sentenceTokens))

        sentenceList.append(sentence)
        sentenceTokenList.append(sentenceTokens)
        sentLengthList.append(sentenceLength)
        bertSentenceIDs.append(tokenizer.convert_tokens_to_ids(sentenceTokens))
        bertMasks.append([1] * (sentenceLength + 1) + [0] * (max_length -1 - sentenceLength ))
        bertSequenceIDs.append([0] * (max_length))

    return (bertSentenceIDs, bertMasks, bertSequenceIDs, sentenceTokenList)

In [0]:
trainBertSentenceIDs, trainBertMasks, trainBertSequenceIDs, trainSentenceTokenList = construct_bert_sentences(df_train, 'movieplot', max_length)

In [0]:
devBertSentenceIDs, devBertMasks, devBertSequenceIDs, devSentenceTokenList = construct_bert_sentences(df_dev, 'movieplot', max_length)

In [0]:
valBertSentenceIDs, valBertMasks, valBertSequenceIDs, valSentenceTokenList = construct_bert_sentences(df_val, 'movieplot', max_length)

In [0]:
X_train = np.array([(trainBertSentenceIDs),(trainBertMasks),(trainBertSequenceIDs)])
X_dev = np.array([(devBertSentenceIDs),(devBertMasks),(devBertSequenceIDs)])
X_val = np.array([(valBertSentenceIDs),(valBertMasks),(valBertSequenceIDs)])

In [0]:
print(X_train.shape)
print(X_dev.shape)
print(X_val.shape)

(3, 46970, 128)
(3, 15438, 128)
(3, 15600, 128)


In [0]:
# Always the same, regardless of the number of labels
np.save('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_train_bert_tokens_' + str(max_length) + '.data', X_train)
np.save('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_dev_bert_tokens_' + str(max_length) + '.data', X_dev)
np.save('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_val_bert_tokens_' + str(max_length) + '.data', X_val)